# Clip Tokenizer



In [ ]:
#@title 1. Install Requirements

print("This will take a couple of minutes...")
!pip install -q --upgrade diffusers[torch]
!pip install -q transformers


In [ ]:
#@title 2. Setup Pipeline

import os
import torch
from transformers import CLIPTokenizer
from transformers import file_utils
from diffusers import StableDiffusionPipeline, DDIMScheduler
# =======================================
# setup pipeline
if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_PATH = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(f'{MODEL_PATH}', revision="fp16", torch_dtype = torch.float16, safety_checker = None)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

tokenizer = CLIPTokenizer.from_pretrained(MODEL_PATH, subfolder="tokenizer",revision="fp16")

# tokenizer/vocab.json files are normally saved in cache e.g.
# /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/xxxxx/tokenizer/vocab.json

# transformers cache directory
cache_dir = file_utils.default_cache_path

# find the vocab file
for root, dirs, files in os.walk(cache_dir):
    for file in files:
        if file == "vocab.json":
            vocab_file_path = os.path.join(root, file)
            print(f"Found vocab file at: {vocab_file_path}")
            break



In [151]:
#@title 3. Setup Tokenize() Function

import json

# =======================================
# for coloring text
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# ===========================================================================

def tokenize(prompt_or_caption):

  with open(vocab_file_path, "r", encoding='utf-8') as f:
    vocab = json.load(f)
    vocab = {v: k for k, v in vocab.items()}

  print(f'vocab size: {tokenizer.vocab_size}')
  print(f'max length: {tokenizer.model_max_length}')

  bos_encoded_input = tokenizer.encode('<|startoftext|>', add_special_tokens=False)
  eos_encoded_input = tokenizer.encode(f'{tokenizer.eos_token}', add_special_tokens=False)
  print(f'{bcolors.OKCYAN}special token (bos): {tokenizer.bos_token}: {bos_encoded_input}{bcolors.ENDC}')
  print(f'{bcolors.OKCYAN}special token (eos): {tokenizer.eos_token}: {eos_encoded_input}{bcolors.ENDC}')

  # print(f'special token (unk): {tokenizer.unk_token}')
  # print(f'special token (pad): {tokenizer.pad_token}\n')

  print(f'{bcolors.OKGREEN}GREEN = Within the 77-Token Limit{bcolors.ENDC}')
  print(f'{bcolors.RED}RED = Outside 77-Token Limit{bcolors.ENDC}\n')

  #inputs = tokenizer(sys.argv[1], padding=True)
  inputs = tokenizer(f'{prompt_or_caption}', padding=True)
  value = sum(inputs["input_ids"])
  value_str = str(value)
  token_data = ''


  #  print the token ids in rows of 10
  row_length = 10
  final_string_ids = '' # final string for the ids
  final_string_words = '' # final string for the words/subwords

  # Find the length of the longest word (for padding)
  max_word_length = max(len(vocab[id]) for id in inputs["input_ids"])

  for i in range(0, len(inputs["input_ids"]), row_length):
    # Extract a row's worth of numbers
    row = inputs["input_ids"][i:i + row_length]

    #  inputs["input_ids"][x] = the ID Numbers
    #  vocab[inputs["input_ids"][x]] = the sub-word

    # Initialize an empty string for the new row
    row_string_ids = ""
    row_string_words = ""

    # Iterate through each token-id in the row, using j as index
    for j, id in enumerate(row):
        # here, id = inputs["input_ids"][j]
        # Calculate the global index of the token
        global_index = i + j

        word = vocab[id]
        padded_word = word.ljust(max_word_length)

        # Determine the color based on the global token index
        if global_index < 77:
            color = bcolors.OKGREEN
        else:
            color = bcolors.RED

        # Append the colored token to the row string
        row_string_ids += f"{color}{id:5d}{bcolors.ENDC}"
        row_string_words += f"{color}{padded_word}{bcolors.ENDC}"

        if j < len(row) - 1:
            row_string_ids += ", "  # Add comma and space between tokens
            row_string_words += ", "  # Add comma and space between tokens


    # Determine if a comma should be added at the end
    end_comma = ',' if i + row_length < len(inputs["input_ids"]) else ''

    # add to final strings
    final_string_ids += row_string_ids + end_comma + '\n'
    final_string_words  += row_string_words + end_comma + '\n'

  print('\n')

  print(final_string_ids)

  print('\n')

  print(final_string_words)




In [152]:
#@title 4. Run

PROMPT_CAPTION = "A charming photo of a small bunny. He is wearing a suit. masterpiece, ultra-quality, hyperrealistic, RAW photo, highly detailed, 4k, medium shot, cinematic photography, natural texture, action shot, XF IQ4, 150MP, 50mm, ISO 1000, 1/250s, natural light"

tokenize(PROMPT_CAPTION)



vocab size: 49408
max length: 77
special token (bos): <|startoftext|>: [49406]
special token (eos): <|endoftext|>: [49407]
GREEN = Within the 77-Token Limit
RED = Outside 77-Token Limit



49406,   320, 12177,  1125,   539,   320,  2442,  9258,   269,   797,
  533,  3309,   320,  3940,   269, 12066,   267,  8118,   268,  3027,
  267,  7997, 16157,   267,  6323,  1125,   267,  5302, 12609,   267,
  275,   330,   267,  8675,  2000,   267, 25602,  2108,   267,  3288,
16505,   267,  1816,  2000,   267, 35274, 19996,   275,   267,   272,
  276,   271,  1246,   267,   276,   271,  2848,   267, 12263,   272,
  271,   271,   271,   267,   272,   270,   273,   276,   271,   338,
  267,  3288,  1395, 49407



<|startoftext|>, a</w>          , charming</w>   , photo</w>      , of</w>         , a</w>          , small</w>      , bunny</w>      , .</w>          , he</w>         ,
is</w>         , wearing</w>    , a</w>          , suit</w>       , .</w>          , masterpiece</w>, ,</w>          , ul